# LSTM
RNN在处理长期依赖（时间序列上距离较远的节点）时会遇到巨大的困难，因为计算距离较远的节点之间的联系时会涉及雅可比矩阵的多次相乘，会造成梯度消失或者梯度膨胀的现象。

循环神经网络除了训练困难，还有一个更严重的问题，那就是短时记忆(Short-term memory).它在处理较长的句子时，往往只能够理解有限长度内的信息，而对于位于较长范围类的有用信息往往不能够很好的利用起来。

![](./images/LSTM2.png)

## 原理

RNN的核心思想是上一个时间戳的状态向量$h_{t-1}$与当前时间戳的输入$x_t$经过线性变换后, 通过激活函数得到新的状态向量$h_{t}$.
LSTM 新增了一个状态向量$𝑪$, 同时引入了门控(Gate)机制，通过门控单元来控制信息的遗忘和刷新，他们包含一个 sigmoid 神经网络层和一个 pointwise 乘法操作.

### 遗忘层门

作用: 将细胞状态中的信息选择性的遗忘, 作用于 LSTM 状态向量𝒄上面，.

操作步骤：该门会读取$h_{t-1}$和$x_t$，输出一个在 0 到 1 之间的数值给每个在细胞状态$C_{t-1}$中的数字。1 表示“完全保留”，0 表示“完全舍弃”。

公式: $$f_t = \sigma(W_f[h_{t-1}, x_t] + b_f)$$

### 输入层门

作用: 将新的信息选择性的记录到细胞状态中, 控制LSTM对输入的接收程度

操作步骤: 

1. sigmoid层决定什么值我们将要更新($i_t$);   
2. tanh 层创建一个新的候选值向量$\tilde{C}_t​$加入到状态中;  
3. 将$c_{t-1}$更新为$c_{t}$, 丢弃需要丢弃的旧状态, 获得需要获得的新状态

公式:
$$
i_t = \sigma(W_i[h_{t-1}, x_t] + b_i) \\
\tilde{C}_t = tanh(W_C[h_{t-1}, x_t] + b_C) \\
C_t = f_t * C_{t-1} + i_t * \tilde C_t
$$


### 输出门层

作用: 确定输出什么值, 内部状态$C_t$不会直接用于输出

操作步骤: 

1. 通过sigmoid 层来确定细胞状态的哪个部分将输出
2. 把细胞状态通过 tanh 进行处理，并将它和 sigmoid 门的输出相乘, 输出确定输出的部分

公式:
$$
o_t = \sigma(W_o[h_{t-1}, x_t] + b_o) \\
h_t = o_t * tanh(C_t)
$$


|输入门控 |遗忘门控 |LSTM行为|
|---|---|---|
|0|1|只是用记忆|
|1|1| 综合输入和记忆|
|0|0|清零记忆|
|1|0|输入覆盖记忆|


### LSTM 使用

In [31]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [32]:
from tensorflow.keras import layers, Input, Model, Sequential, datasets

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        print(gpu)
except RuntimeError as e:
    print(e)

In [33]:
x = tf.random.normal([2, 80, 100])  # 
xt = x[:, 0, :]  # 第一个单词, 第一个时间戳的输入

cell = layers.LSTMCell(64)  # 与SimpleRNNCell 类似
state = [tf.zeros([2, 64]), tf.zeros([2, 64])]  # 初始状态h0,C0
out, state = cell(xt, state)

In [34]:
cell?

In [10]:
out.shape

TensorShape([2, 64])

In [11]:
cell.get_config()

{'name': 'lstm_cell_1',
 'trainable': True,
 'dtype': 'float32',
 'units': 64,
 'activation': 'tanh',
 'recurrent_activation': 'sigmoid',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'unit_forget_bias': True,
 'kernel_regularizer': None,
 'recurrent_regularizer': None,
 'bias_regularizer': None,
 'kernel_constraint': None,
 'recurrent_constraint': None,
 'bias_constraint': None,
 'dropout': 0.0,
 'recurrent_dropout': 0.0,
 'implementation': 2}

In [12]:
w_xh,w_hh, b =  cell.trainable_variables
w_xh.shape, w_hh.shape, b.shape  # 4 个部分堆叠

(TensorShape([100, 256]), TensorShape([64, 256]), TensorShape([256]))

In [13]:
for xt in tf.unstack(x, axis=1):
    out, state = cell(xt, state)

In [14]:
# LSTM层
lstm_layer = layers.LSTM(64)

out = lstm_layer(x)
out.shape

TensorShape([2, 64])

In [38]:
lstm_layer1 = layers.LSTM(64, return_sequences=True)

out = lstm_layer1(x)
out.shape

TensorShape([2, 80, 64])

In [ ]:
x = tf.random.normal([2, 1, 100])  # 
xt = x[:, 0, :]  # 第一个单词, 第一个时间戳的输入

cell = tf.keras.layers.LSTM(64, return_state=True)  
state = [tf.zeros([2, 64]), tf.zeros([2, 64])]  # 初始状态h0,C0
out, state0, state1 = cell(x, state)

In [37]:
out[0], out[1]

(<tf.Tensor: id=12711, shape=(80, 64), dtype=float32, numpy=
 array([[ 0.02453458,  0.00537144,  0.16522579, ..., -0.14636117,
          0.12870291, -0.19787529],
        [-0.14883734, -0.06304549,  0.0218755 , ..., -0.07071137,
          0.00717948, -0.03601092],
        [-0.09798   , -0.17283341,  0.12789893, ..., -0.20422818,
          0.19600664, -0.10933438],
        ...,
        [-0.23750345, -0.21065132, -0.22160155, ...,  0.02451491,
         -0.22423549, -0.08433109],
        [-0.24251884, -0.0784284 , -0.01251083, ...,  0.22695696,
         -0.28828096, -0.11059391],
        [-0.22755438, -0.1300424 , -0.26687545, ...,  0.03499105,
         -0.18308741, -0.06962312]], dtype=float32)>,
 <tf.Tensor: id=12715, shape=(80, 64), dtype=float32, numpy=
 array([[ 0.12468018, -0.16513684,  0.13801843, ...,  0.20605457,
         -0.09332453, -0.02846486],
        [ 0.12871115, -0.09259171,  0.15879299, ...,  0.27870795,
          0.02170868,  0.11957248],
        [ 0.0965796 , -0.432499

In [15]:
# 简单堆叠LSTM层
lstm_net = Sequential([
    layers.LSTM(units=64, return_sequences=True),
    layers.LSTM(units=64),
])
lstm_net = lstm_net(x)

###  使用LSTM进行情感分类

In [16]:
BATCH_SIZE = 128
TOTAL_WORDS = 10000  # 词汇表大小
MAX_REVIEW_LEN = 80  # 句子长度
EMBEDDING_LEN = 100  # 词向量长度

In [17]:
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=TOTAL_WORDS)

In [18]:
word_index = datasets.imdb.get_word_index()

pre_10 = list(word_index.items())[:10]
for item in pre_10:  
    print(item)  # 单词-数字

('fawn', 34701)
('tsukino', 52006)
('nunnery', 52007)
('sonja', 16816)
('vani', 63951)
('woods', 1408)
('spiders', 16115)
('hanging', 2345)
('woody', 2289)
('trawling', 52008)


In [19]:
# 添加标志位
word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0  # 表示填充
word_index["<START>"] = 1  # 表示起始
word_index["<UNK>"] = 2  # 表示未知单词
word_index["<UNUSED>"] = 3

# 翻转
index_word = dict([(v, k) for k, v in word_index.items()]) 

In [20]:
def decode_review(text):
    # 数字序列 -> 文本
    return ' '.join([index_word.get(i, '?') for i in text])

In [21]:
decode_review(X_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
# 截断填充(前部分) 成等长的序列
X_train = pad_sequences(X_train, maxlen=MAX_REVIEW_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_REVIEW_LEN)

In [24]:
X_train.shape

(25000, 80)

In [25]:
train_db = tf.data.Dataset.from_tensor_slices(  # 舍弃最后一组 
    (X_train, y_train)).shuffle(1000).batch(BATCH_SIZE, drop_remainder=True)
test_db = tf.data.Dataset.from_tensor_slices(
    (X_test, y_test)).shuffle(1000).batch(BATCH_SIZE, drop_remainder=True)

In [26]:
def load_embed(path):
    # 建立映射关系: 单词: 词向量(长度50))
    embedding_map = {}
    with open(path, encoding='utf8') as f:
        for line in f.readlines():
            l = line.split()
            word = l[0]
            coefs = np.asarray(l[1:], dtype='float32')
            embedding_map[word] = coefs
    return embedding_map

In [27]:
embedding_map = load_embed('glove.6B.100d.txt')
print('Found %s word vectors.' % len(embedding_map))

Found 400000 word vectors.


In [28]:
# 预训练
# 将 单词序号-> 单词向量
num_words = min(TOTAL_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_LEN))

applied_vec_count = 0
for word, i in word_index.items():
    if i >= TOTAL_WORDS:
        continue
    # 根据glove.6B.100d 将单词转为词向量
    embedding_vector = embedding_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        applied_vec_count += 1
print(applied_vec_count, embedding_matrix.shape)

9793 (10000, 100)


In [29]:
class MyLSTMRNN(Model):
    def __init__(self, units):
        super().__init__()
        # 初始状态向量
#         self.state0 = [tf.zeros([BATCH_SIZE, units])]
#         self.state1 = [tf.zeros([BATCH_SIZE, units])]
        # 词嵌入层
        self.embedding = layers.Embedding(TOTAL_WORDS, EMBEDDING_LEN,
                                          input_length=MAX_REVIEW_LEN,
                                          weights=[embedding_matrix],
                                         trainable=False
                                         )
        # RNNCell
#         self.runcell0 = layers.SimpleRNNCell(units, dropout=0.5)
#         self.runcell1 = layers.SimpleRNNCell(units, dropout=0.5)
        # RNN layer
        self.rnn = Sequential([
            layers.LSTM(units, dropout=0.5, return_sequences=True),
            layers.LSTM(units, dropout=0.5)
        ])
        # 分类层
        self.out_layer = Sequential([
            layers.Dense(32, activation='relu'),
            layers.Dropout(rate=0.5),
            layers.Dense(1, activation='sigmoid')
        ])
        
    
    def call(self, inputs, training=None):
        x = self.embedding(inputs)
#         state0, state1 = self.state0, self.state1
#         for word in tf.unstack(x, axis=1):
#             out0, state0 = self.runcell0(word, state0, training)
#             out1, state1 = self.runcell1(out0, state1, training)
        out1 = self.rnn(x)
        # 最末层 最后一个时间戳的输出
        out = self.out_layer(out1, training)
        return out

In [30]:
model = MyLSTMRNN(64)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build((None, MAX_REVIEW_LEN))
model.summary()

Model: "my_lstmrnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1000000   
_________________________________________________________________
sequential_1 (Sequential)    multiple                  75264     
_________________________________________________________________
sequential_2 (Sequential)    multiple                  2113      
Total params: 1,077,377
Trainable params: 77,377
Non-trainable params: 1,000,000
_________________________________________________________________


In [ ]:
hist = model.fit(train_db, epochs=20, validation_data=test_db)

In [ ]:
plt.plot(hist.history['loss'], label='train_loss')
plt.plot(hist.history['val_loss'], label='test_loss')
plt.legend()

In [ ]:
plt.plot(hist.history['accuracy'], label='train_accuracy')
plt.plot(hist.history['val_accuracy'], label='test_accuracy')
plt.legend()

# GRU
LSTM 不容易出现梯度弥散现象。但是LSTM 结构相对较复杂，计算代价较高，模型参数量较大。
门控循环网络(Gated Recurrent Unit，GRU), 是应用最广泛的LSTM简化版本. 将忘记门和输入门合成了一个单一的更新门, 内部状态向量和输出向量合并，统一为状态向量h.
![](./images/LSTM12.png)

- 复位门(Reset Gate): 控制上一个时间戳的状态$h_{t-1}$进入GRU 的量;
- 更新门(Update Gate): 控制上一时间戳的状态$h_{t-1}$和新输入$\tilde h_t$对新状态向量$h_t$的影响程度.

In [ ]:
h = [tf.zeros([2, 64])]
cell = layers.GRUCell(64)

for xt in tf.unstack(x, axis=1):
    out, h = cell(xt, h)
    
out.shape

In [ ]:
class MyGRURNN(Model):
    def __init__(self, units):
        super().__init__()
        # 初始状态向量
#         self.state0 = [tf.zeros([BATCH_SIZE, units])]
#         self.state1 = [tf.zeros([BATCH_SIZE, units])]
        # 词嵌入层
        self.embedding = layers.Embedding(TOTAL_WORDS, EMBEDDING_LEN,
                                          input_length=MAX_REVIEW_LEN,
                                          weights=[embedding_matrix],
                                         trainable=False
                                         )
        # RNNCell
#         self.runcell0 = layers.SimpleRNNCell(units, dropout=0.5)
#         self.runcell1 = layers.SimpleRNNCell(units, dropout=0.5)
        # RNN layer
        self.rnn = Sequential([
            layers.GRU(units, dropout=0.5, return_sequences=True),
            layers.GRU(units, dropout=0.5)
        ])
        # 分类层
        self.out_layer = Sequential([
            layers.Dense(32, activation='relu'),
            layers.Dropout(rate=0.5),
            layers.Dense(1, activation='sigmoid')
        ])
        
    
    def call(self, inputs, training=None):
        x = self.embedding(inputs)
#         state0, state1 = self.state0, self.state1
#         for word in tf.unstack(x, axis=1):
#             out0, state0 = self.runcell0(word, state0, training)
#             out1, state1 = self.runcell1(out0, state1, training)
        out1 = self.rnn(x)
        # 最末层 最后一个时间戳的输出
        out = self.out_layer(out1, training)
        return out

In [ ]:
model = MyGRURNN(64)
model.build((None, MAX_REVIEW_LEN))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train_db, epochs=20, validation_data=test_db)

In [ ]:
plt.plot(hist.history['loss'], label='train_loss')
plt.plot(hist.history['val_loss'], label='test_loss')
plt.legend()

In [ ]:
plt.plot(hist.history['accuracy'], label='train_accuracy')
plt.plot(hist.history['val_accuracy'], label='test_accuracy')
plt.legend()

# 双向RNN
Bidirectional RNNs(双向网络)将两层RNNs叠加在一起，当前时刻输出(第t步的输出)不仅仅与之前序列有关，还与之后序列有关。
![](./images/figure_6.6.2_1.png)

In [5]:
layers.Bidirectional?